# Evaluación sobre verdad de campo original

In [ ]:
import os
import json
import joblib
import numpy as np
import pandas as pd
from glob import glob
from sqlite3 import connect
from tqdm.notebook import tqdm
from sklearn.ensemble import RandomForestClassifier

from utilities import *

In [ ]:
sqlite_files = glob('../data/selection/*.sqlite')

data = pd.DataFrame()

for sf in sqlite_files:
    cnx = connect(sf)
    df = pd.read_sql_query("SELECT * FROM output", cnx)
    data = pd.concat([data, df], ignore_index=True)

In [ ]:
data.head()

In [ ]:
model = joblib.load('../model/randomforest_iter_01.joblib')

In [ ]:
X = data.filter(regex='band_').to_numpy()
y = data['id'].to_numpy()

In [ ]:
y_hat = model.predict_proba(X)